In [1]:
from PIL import Image
import numpy as np
from pathlib import Path
from sklearn.preprocessing import OneHotEncoder
file_paths = [] # an empty list to store the file paths
labels = [] # an empty list to store the labels
for path in Path('fer_ckplus_kdef').rglob('*.png'): # loop through all png files in the dataset folder and its subfolders
    file_paths.append(str(path)) # append the file path as a string to the list
    labels.append(path.parent.name) # append the folder name as the label to the list
print(len(file_paths))
enc = OneHotEncoder(sparse=False)
labels = np.array(labels).reshape(-1, 1)
labels = enc.fit_transform(labels)
print(len(labels))
print(labels)

32854
32854
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


/home/default/miniconda3/envs/mne/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
from sklearn import model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    file_paths, labels, test_size=0.125, random_state=42, shuffle=True
)

In [3]:
import tensorflow as tf
import math
class ImageSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.x))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]

        return np.array([np.array(Image.open(file_name)) / 255.0 for file_name in batch_x]), np.array(batch_y)

2023-09-07 16:32:39.832332: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 16:32:40.938896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Dropout,Flatten,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras import optimizers
num_classes = 8
def AlexNet():
    model = Sequential()
    clear_session()
    #Conv2D 1
    model.add(Input((224, 224, 1)))
    model.add(Conv2D(96, 11, strides=(4, 4), padding='valid', activation='relu')) 
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
    model.add(BatchNormalization())
    #Conv2D 2
    model.add(Conv2D(256, 5, strides=(1, 1), padding='same', activation='relu')) 
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
    model.add(BatchNormalization())
    #Conv2D 3 4 5
    model.add(Conv2D(384, 3, strides=(1, 1), padding='same', activation='relu')) 
    model.add(Conv2D(384, 3, strides=(1, 1), padding='same', activation='relu')) 
    model.add(Conv2D(256, 3, strides=(1, 1), padding='same', activation='relu')) 
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
    model.add(BatchNormalization())
    #Dense 1
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    #Dense 2
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    #Dense 3
    model.add(Dense(num_classes, activation='softmax'))
    model.compile('sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = AlexNet()
model.summary()

In [28]:
from keras.callbacks import ReduceLROnPlateau
training_generator = ImageSequence(x_train, y_train, 128)
validation_generator = ImageSequence(x_test, y_test, 128)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
model.fit(training_generator, epochs=50, validation_data=validation_generator, callbacks=[reduce_lr]) 

Epoch 1/50
225/225 [==============================] - 30s 123ms/step - loss: 2.0769 - accuracy: 0.2664 - val_loss: 1.7997 - val_accuracy: 0.3002 - lr: 0.0100
Epoch 2/50
225/225 [==============================] - 24s 107ms/step - loss: 1.6941 - accuracy: 0.3672 - val_loss: 1.5930 - val_accuracy: 0.4076 - lr: 0.0100
Epoch 3/50
225/225 [==============================] - 24s 108ms/step - loss: 1.5237 - accuracy: 0.4321 - val_loss: 1.4187 - val_accuracy: 0.4787 - lr: 0.0100
Epoch 4/50
225/225 [==============================] - 24s 108ms/step - loss: 1.4062 - accuracy: 0.4746 - val_loss: 1.4356 - val_accuracy: 0.4599 - lr: 0.0100
Epoch 5/50
225/225 [==============================] - 24s 108ms/step - loss: 1.3125 - accuracy: 0.5095 - val_loss: 1.3308 - val_accuracy: 0.5116 - lr: 0.0100
Epoch 6/50
225/225 [==============================] - 24s 108ms/step - loss: 1.2271 - accuracy: 0.5406 - val_loss: 1.2620 - val_accuracy: 0.5398 - lr: 0.0100
Epoch 7/50
225/225 [==============================] 

In [ ]:
model.save('fer_ckplus_kdef_alexnet')